In [ ]:
import numpy as np

# take in left and right wheel distance (traveled on the ground, not
# raw wheel rotations) and the distance between the wheels
# return the dx, dy, and heading change at t_1 for the t_0,
# body coordinate frame, e.g. given the body is in one location
# before the wheels move where will it be relative to that after they move?
# this is modelled as a circular motion, there is a spin center defined
# by the relative motion of the wheels, it is off at infinity when traveling
# in a straight line
def wheel_distance_to_state_change(
    left_distance: float,
    right_distance: float,
    wheel_base: float,
) -> (float, float, float):
    theta = (right_distance - left_distance) / wheel_base
    center_distance = (right_distance + left_distance) / 2.0
    
    dx = center_distance * np.sinc(theta / np.pi)
    dy = dx * np.tan(theta / 2.0)
    
    return (dx, dy, theta, center_distance)

t0 = 0.0
t1 = 2.0

wheel_base = 0.1

dts = [0.00011, 0.001, 0.01, 0.017, 0.045, 0.09]

import matplotlib
import matplotlib.pyplot as plt

fig0 = plt.figure(figsize=(12, 12))
ax0 = fig0.add_subplot(1, 1, 1)

fig1 = plt.figure(figsize=(14, 5))
ax1 = fig1.add_subplot(1, 1, 1)
plt.ylabel('left/right distance')

fig2 = plt.figure(figsize=(14, 5))
ax2 = fig2.add_subplot(1, 1, 1)
plt.ylabel('x')

fig3 = plt.figure(figsize=(14, 5))
ax3 = fig3.add_subplot(1, 1, 1)
plt.ylabel('y')

fig4 = plt.figure(figsize=(14, 5))
ax4 = fig4.add_subplot(1, 1, 1)
plt.ylabel('theta/heading')

for dt in dts:
    ts = np.arange(t0, t1, dt)
    num = ts.shape[0]
    print(f"{dt} {num}")
    ramp = np.arange(0, num) / num
    dist_left = 14.0 * ramp
    dist_left += 0.2 * np.sin(ts * 2.569) * ramp
    dist_left -= dist_left[0]
    # dist_right = abs(np.sin(ts * 1.9))
    dist_right = 14.0 * ramp
    dist_right -= dist_right[0]

    # TODO(lucasw) vectorize

    # initial state
    x = 0.0
    y = 0.0
    theta = 0.0
    xs = []
    ys = []
    cs = []
    thetas = []
    last_l = dist_left[0]
    last_r = dist_right[0]
    dist_traveled = 0.0
    for ind, (t, l, r) in enumerate(zip(ts, dist_left, dist_right)):
        dl = l - last_l
        dr = r - last_r
        last_l = l
        last_r = r
        
        dx_in_body, dy_in_body, dtheta, c_dist = wheel_distance_to_state_change(dl, dr, wheel_base)
        dist_traveled += c_dist
        
        x += dx_in_body * np.cos(theta) - dy_in_body * np.sin(theta)
        y += dx_in_body * np.sin(theta) + dy_in_body * np.cos(theta)
        theta += dtheta

        cs.append(dist_traveled)
        xs.append(x)
        ys.append(y)
        thetas.append(theta)

    print(f"dt: {dt}s, distance traveled: {dist_traveled:.6}")
    ax0.plot(xs, ys, '-.')
    ax1.plot(ts, dist_left, '.')
    ax1.plot(ts, dist_right, '.')
    ax2.plot(ts, xs, '.')
    ax3.plot(ts, ys, '.')
    ax4.plot(ts, thetas, '.')
    
ax0.plot(0.0, 0.0, 'o')
ax0.plot(xs[-1], ys[-1], 'o')
ax0.axis('equal')